In [80]:
%matplotlib inline
import gspread
from gspread.models import Spreadsheet
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from bs4 import BeautifulSoup
import requests
from IPython.display import display, Markdown
import pandas as pd
from urllib.parse import urljoin
from io import StringIO, BytesIO
import re
import numpy as np
import seaborn as sns
import plotly.express as px

In [2]:
# get google sheet to read and store data
gc = gspread.service_account(
    filename="/Users/grahamflemingthomson/Downloads/grahamflemingthomson-164514-04e450fcd324.json"
)
sh = gc.open_by_key(
    "1OqoSsf5YrsJWRWK1WouiwHeTP_prGVmsP7yURgjQnwo"
)

def write_to_worksheet(sheet: Spreadsheet, worksheet_name: str, df: pd.DataFrame) -> None:
    worksheet_dict = {ws.title: i for i, ws in enumerate(sheet.worksheets())}
    if worksheet_name not in worksheet_dict:
        sheet.add_worksheet(title=worksheet_name, rows=1, cols=1)
        worksheet_dict = {ws.title: i for i, ws in enumerate(sheet.worksheets())}
    set_with_dataframe(
        worksheet=sheet.get_worksheet(worksheet_dict.get(worksheet_name)), 
        dataframe=df
    )
    return None

In [3]:
# globals
ALL_POSITIONS = {'LB', 'WR', 'RB', 'S', 'CB', 'DE', 'QB', 'PN', 'PK', 'TE', 'Off', 'DT'}
POSITIONS = ALL_POSITIONS - {'Off'}
SEASON_YEARS = {2019, 2020}
BASE_URL = "https://www58.myfantasyleague.com/"

In [4]:
def get_stats(
    snapshot_year: int = 2021,
    year: int = 2020, 
    count: int = 500, 
    start_week: int = 1, 
    end_week: int = 16, 
    category: str = "overall", 
    position: str = "*", 
    display: str = "points",
    search_type: str = "BASIC",
    team: str = "*",
    l: int = 46381,
    append_player_paths: bool = True
    ) -> pd.DataFrame:
    
    param_dict = {
        "L": l,
        "SEARCHTYPE": search_type,
        "COUNT": count,
        "YEAR": year,
        "START_WEEK": start_week,
        "END_WEEK": end_week,
        "CATEGORY": category,
        "POSITION": position,
        "DISPLAY": display,
        "TEAM": team
    }
    
    stats_url = urljoin(BASE_URL, f"{snapshot_year}/top")
    
    req = requests.get(stats_url, params=param_dict)
    resp = req.content
    
    stats_df = pd.read_html(BytesIO(resp)).pop(1)
    # rename columns
    stats_df.columns = [
        'rank', 'player_name', 'total_points', 'average_points', 'week_1_pts', 
        'week_2_pts', 'week_3_pts', 'week_4_pts', 'week_5_pts', 'week_6_pts', 
        'week_7_pts', 'week_8_pts', 'week_9_pts', 'week_10_pts', 'week_11_pts', 
        'week_12_pts', 'week_13_pts', 'week_14_pts', 'week_15_pts', 'week_16_pts', 
        'week_1_opponent', 'owner', 'bye_week', 'salary', 'years'
    ]
    stats_df["year"] = year
    stats_df["position"] = stats_df["player_name"].apply(lambda x: x.split()[-1].strip())
    
    if append_player_paths:
        soup = BeautifulSoup(resp, 'html.parser')
        players_paths_df = pd.DataFrame(
            [(a.text, a.get("href")) 
             for a in soup.find_all('a') 
             if a.get("href", "NULL").startswith("player?L") 
             and a.get("class") is not None],
            columns=["player_name", "player_path"]
        )
        stats_df = stats_df.merge(players_paths_df, on="player_name", how="left")
    
    return stats_df


def get_player_demos(player_name: str, player_path: str, snapshot_year: int = 2021) -> pd.DataFrame:
    try:
        demos = pd.read_html(urljoin(BASE_URL, f"{snapshot_year}/{player_path}")).pop(2)
        demos = demos.set_index(0).transpose().drop(2).rename(columns={"Experience:": "experience"})
        demos["height"], demos["weight"] = demos["Height/Weight:"].apply(lambda x: x.split(" / ")).values[0]
        demos["dob"], demos["age"] = demos["DOB/Age:"].apply(lambda x: x.split(" / ")).values[0]
        demos["player_name"] = player_name
        return demos[['height', 'weight', 'dob', 'age', 'experience', 'player_name']]
    # lazy exception 'handling'
    except:
        return pd.DataFrame()


def append_player_demos(stats_df: pd.DataFrame) -> pd.DataFrame:
    demo_df = pd.concat([df for df in
                stats_df.apply(
                    lambda row: get_player_demos(
                        player_name=row.player_name, 
                        player_path=row.player_path
                    ), axis=1).tolist()
               if df.shape[0] > 0], ignore_index=True)
    return stats_df.merge(demo_df, on="player_name", how="left")

In [5]:
all_dfs = []

for sy in SEASON_YEARS:
    for p in ALL_POSITIONS:
        p_stats = get_stats(year=sy, position=p)
        all_dfs.append(p_stats)

In [6]:
all_df = pd.concat(all_dfs, ignore_index=True)
all_df.head()

,rank,player_name,total_points,average_points,week_1_pts,week_2_pts,week_3_pts,week_4_pts,week_5_pts,week_6_pts,...,week_15_pts,week_16_pts,week_1_opponent,owner,bye_week,salary,years,year,position,player_path
0,1.0,"Watt, T.J. PIT LB",176.2,11.747,5.0,14.0,13.7,11.50,12.50,6.50,...,14.0,14.0,@BUF,The Brandon Administration,7,"$5,495,000",5,2019,LB,player?L=46381&P=13214
1,2.0,"Hicks, Jordan ARI LB",175.4,11.693,15.5,11.0,8.0,20.00,8.50,6.50,...,11.5,11.5,@TEN,Cucamonga Cracker Killers,12,"$450,000",1,2019,LB,player?L=46381&P=12287
2,3.0,"Schobert, Joe CLE LB",172.5,11.500,6.5,17.5,10.1,20.00,13.00,7.00,...,4.0,7.0,@HOU,Dababetes Type II,7,"$4,950,000",1,2019,LB,player?L=46381&P=12768
3,4.0,"Leonard, Darius IND LB",168.7,14.058,7.5,13.5,NaN,NaN,NaN,B,...,9.0,8.0,SEA,WYLD STALLYNS,6,"$890,000",2,2019,LB,player?L=46381&P=13733
4,5.0,"Jones, Chandler ARI LB",167.0,11.133,11.5,2.5,18.5,-1.70,6.00,4.00,...,10.5,32.0,@TEN,The Poop Swatches,12,"$5,000,000",3,2019,LB,player?L=46381&P=10753


In [8]:
write_to_worksheet(sheet=sh, worksheet_name="2019_top_players", df=all_df[all_df["year"] == 2019])
write_to_worksheet(sheet=sh, worksheet_name="2020_top_players", df=all_df[all_df["year"] == 2020])

In [9]:
distinct_players = all_df[["player_name", "player_path"]].drop_duplicates()
distinct_player_demos = append_player_demos(stats_df=distinct_players)

In [19]:
all_df.drop(["player_path_y", "height", "weight", "dob", "age", "experience"], axis=1, inplace=True)

In [21]:
all_df = all_df.rename(columns={"player_path_x": "player_path"})

In [22]:
all_df = all_df.merge(distinct_player_demos, on=["player_name", "player_path"], how="left")
all_df.head()

,rank,player_name,total_points,average_points,week_1_pts,week_2_pts,week_3_pts,week_4_pts,week_5_pts,week_6_pts,...,salary,years,year,position,player_path,height,weight,dob,age,experience
0,1.0,"Watt, T.J. PIT LB",176.2,11.747,5.0,14.0,13.7,11.50,12.50,6.50,...,"$5,495,000",5,2019,LB,player?L=46381&P=13214,"6'4""",252lbs,"Oct 11, 1994",26,5 years
1,2.0,"Hicks, Jordan ARI LB",175.4,11.693,15.5,11.0,8.0,20.00,8.50,6.50,...,"$450,000",1,2019,LB,player?L=46381&P=12287,"6'1""",236lbs,"Jun 27, 1992",28,7 years
2,3.0,"Schobert, Joe CLE LB",172.5,11.500,6.5,17.5,10.1,20.00,13.00,7.00,...,"$4,950,000",1,2019,LB,player?L=46381&P=12768,"6'1""",245lbs,"Nov 6, 1993",27,6 years
3,4.0,"Leonard, Darius IND LB",168.7,14.058,7.5,13.5,NaN,NaN,NaN,B,...,"$890,000",2,2019,LB,player?L=46381&P=13733,"6'2""",230lbs,"Jul 27, 1995",25,4 years
4,5.0,"Jones, Chandler ARI LB",167.0,11.133,11.5,2.5,18.5,-1.70,6.00,4.00,...,"$5,000,000",3,2019,LB,player?L=46381&P=10753,"6'5""",255lbs,"Feb 27, 1990",31,10 years


In [44]:
def convert_height_to_inches(height):
    if pd.isna(height):
        return height
    return (np.array(re.findall("\d+", height), dtype=int) * np.array([12, 1])).sum()

def convert_weight_to_lbs(weight):
    if pd.isna(weight):
        return weight
    return np.array(re.findall("\d+", weight), dtype=int).sum()

def parse_digits_to_float(col):
    if pd.isna(col):
        return col
    return float("".join(re.findall("\d+", col)))

In [50]:
all_df["height_inches"] = all_df[f"height"].apply(convert_height_to_inches)
all_df["weight_lbs"] = all_df[f"weight"].apply(convert_weight_to_lbs)
all_df["experience_years"] = all_df[f"experience"].apply(convert_weight_to_lbs)
all_df["salary"] = all_df["salary"].apply(parse_digits_to_float)
all_df["dob"] = pd.to_datetime(all_df["dob"])
cols_to_numeric = ["age", "total_points", "average_points"]
for c in cols_to_numeric:
    all_df[c] = all_df[c].astype(float)

In [51]:
all_df.head()

,rank,player_name,total_points,average_points,week_1_pts,week_2_pts,week_3_pts,week_4_pts,week_5_pts,week_6_pts,...,position,player_path,height,weight,dob,age,experience,height_inches,weight_lbs,experience_years
0,1.0,"Watt, T.J. PIT LB",176.2,11.747,5.0,14.0,13.7,11.50,12.50,6.50,...,LB,player?L=46381&P=13214,"6'4""",252lbs,1994-10-11,26.0,5 years,76.0,252.0,5.0
1,2.0,"Hicks, Jordan ARI LB",175.4,11.693,15.5,11.0,8.0,20.00,8.50,6.50,...,LB,player?L=46381&P=12287,"6'1""",236lbs,1992-06-27,28.0,7 years,73.0,236.0,7.0
2,3.0,"Schobert, Joe CLE LB",172.5,11.500,6.5,17.5,10.1,20.00,13.00,7.00,...,LB,player?L=46381&P=12768,"6'1""",245lbs,1993-11-06,27.0,6 years,73.0,245.0,6.0
3,4.0,"Leonard, Darius IND LB",168.7,14.058,7.5,13.5,NaN,NaN,NaN,B,...,LB,player?L=46381&P=13733,"6'2""",230lbs,1995-07-27,25.0,4 years,74.0,230.0,4.0
4,5.0,"Jones, Chandler ARI LB",167.0,11.133,11.5,2.5,18.5,-1.70,6.00,4.00,...,LB,player?L=46381&P=10753,"6'5""",255lbs,1990-02-27,31.0,10 years,77.0,255.0,10.0


In [52]:
write_to_worksheet(sheet=sh, worksheet_name="2019_top_players", df=all_df[all_df["year"] == 2019])
write_to_worksheet(sheet=sh, worksheet_name="2020_top_players", df=all_df[all_df["year"] == 2020])